In [2]:
import os
from pyspark import SparkContext
sc = SparkContext(appName="search", master=os.environ['MASTER'])
import pysam

In [3]:
from pyspark.sql import SQLContext, Row
sqlCtx = SQLContext(sc)

In [4]:
#read bam file and put identicalID, tlen and read seq into sparkContext
filename = "../../../GenomeData/NA21144.chrom20.ILLUMINA.bwa.GIH.low_coverage.20130415.bam"
samfile = pysam.AlignmentFile(filename, 'rb')
bf = sc.parallelize([' '])
tlenList = []
for ite in samfile.fetch():
    if abs(ite.tlen) > 0:
        tlenList.append(("NA21144", ite.tlen, ite.seq))

lines = sc.parallelize(tlenList)  

#populate data into sqlContext
seqVar = lines.map(lambda p: Row(iid=p[0], tlen=p[1], seq=p[2]))
seqVarTable = sqlCtx.inferSchema(seqVar)  # pop memory exceeds error here
seqVarTable.registerAsTable("seqVar")



#seqVarTable.printSchema()
#print seqVarTable.collect()
#seqList = sqlCtx.sql("SELECT seq from seqVar")


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Serialized task 0:0 was 130910235 bytes, which exceeds max allowed: spark.akka.frameSize (10485760 bytes) - reserved (204800 bytes). Consider increasing spark.akka.frameSize or using broadcast variables for large values.
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1203)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1192)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:1191)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:47)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1191)
	at org.apache.spark.scheduler.DAGScheduler$$anon